# Find definitions of lemmas in arguments of tactics

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import pandas as pd

DATA_DIRECTORY = Path("./data/")  # change as necessary

## 1. Load tactics raw data

In [3]:
tactic_state_goal_raw = pd.read_json(
    DATA_DIRECTORY / "raw_traced_data" / "tactic_state_goal.jsonl",
    orient='records',
    lines=True
)
tactic_state_raw = pd.read_json(
    DATA_DIRECTORY / "raw_traced_data" / "tactic_state.jsonl",
    orient="records",
    lines=True 
)

args = pd.read_json(
    DATA_DIRECTORY / "extracted_proof_data" / "args.jsonl", 
    orient="records",
    lines=True
)

tactic_param_value = pd.read_json(
    DATA_DIRECTORY / "raw_traced_data" / "tactic_param_value.jsonl",
    orient='records',
    lines=True
)

tactics = pd.read_json(
    DATA_DIRECTORY / "extracted_proof_data" / "tactics.jsonl", 
    orient='records',
    lines=True
)

## 2. Join goal and tactic_instance_key

In [4]:
goals = tactic_state_goal_raw.copy()
goals = goals[goals['ix'] == 0]

# set unique index
goals['tactic_state_key'] = goals['filename'] + ":" + goals['tactic_state']
goals = goals[['tactic_state_key', 'goal_pp']]
goals = goals.set_index('tactic_state_key')

states = tactic_state_raw.copy()
# states = states[states['before_after'] == 'before']
states['tactic_state_key'] = states['filename'] + ":" + states['key']
states['tactic_instance_key'] = states['filename'] + ":" + states['tactic_instance']
states['tactic_key'] = states['tactic_instance_key'].apply(lambda k: ":".join(k.split(":")[:-1]))
states = states[['tactic_state_key', 'tactic_key', 'tactic_instance_key', 'before_after', 'tactic_instance']]
states = states.set_index('tactic_state_key')

print(len(goals), len(states))
goals = states.join(goals)
goals = goals.set_index('tactic_key')
goals = goals[goals['before_after'] == 'before'].drop(columns='before_after')
goals.shape

481088 481088


(240974, 3)

## 3. Get tactic arguments

In [5]:
# the lean internal values are indexed by the starting position of
# the parameter argument.  Unfortionately, because of zero length
# arguments, two different arguements can have the same starting
# position.  We add a cumulative count to address this.
params = tactic_param_value.copy()
params['key'] = params['filename'] + ":" + params['key']
params['cnt'] = params.groupby('key').cumcount()
params['key2'] = params['key'] + ":" + params['cnt'].astype(str)
params = params.set_index('key2')
params = params[['reflected_expr_pp']]

args_processed = args.copy()
# args has duplicates since there is a 
# line for each tactic execution instance
args_processed = args_processed.drop_duplicates()  
args_processed['cnt'] = args_processed.groupby('key').cumcount()
args_processed['key2'] = args_processed['key'] + ":" + args_processed['cnt'].astype(str)
args_processed = args_processed.set_index('key2')
args_processed = args_processed.rename(columns={'code_string': 'human_tactic_arg'})
args_processed = args_processed[['parent_key', 'index', 'human_tactic_arg']]

args_params = args_processed.join(params).reset_index()[['parent_key', 'index', 'human_tactic_arg', 'reflected_expr_pp']]
args_params = args_params.set_index('parent_key')
args_params = args_params.pivot_table(
    values=['human_tactic_arg', 'reflected_expr_pp'],
    index='parent_key',
    columns='index',
    aggfunc='first'
)

## 4. Join arguments with tactics and goals

In [6]:
df4 = tactics.copy()
df4['tactic_state_key'] = df4['filename'] + ':' + df4['trace_key']
df4 = df4.set_index('tactic_state_key')
df4 = goals.join(df4)
df4 = df4[df4['class'] == "named"]
df4 = df4[['key', 'code_string', 'goal_pp', 'tactic_instance']]
df4 = df4.rename(columns={'code_string': ('proof_state', 'tactic'),
                          'goal_pp': ('proof_state', 'goal_pp'),
                          'tactic_instance': ('proof_state', 'tactic_instance')})
df4 = df4.set_index('key')

parsed_tactics = df4.join(args_params)
parsed_tactics.columns = pd.MultiIndex.from_tuples(parsed_tactics.columns)

/var/folders/r0/vbnz41hd1vqb29w40dsgy0lc0000gq/T/ipykernel_77130/3099174068.py:12: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  parsed_tactics = df4.join(args_params)


## 4. Join metadata with open namespaces

In [7]:
data_and_metadata = pd.read_csv(DATA_DIRECTORY / "cleaned_training_data" / "data_and_metadata.csv")
data_and_metadata['key'] = data_and_metadata['filename'] + ':' + data_and_metadata.line.astype(str) + ':' + data_and_metadata.column.astype(str)
data_and_metadata = data_and_metadata.set_index('key')

data = parsed_tactics.join(data_and_metadata)
data = data[data.goal_pp == data[('proof_state', 'goal_pp')]]
data.shape

/var/folders/r0/vbnz41hd1vqb29w40dsgy0lc0000gq/T/ipykernel_77130/501957780.py:5: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  data = parsed_tactics.join(data_and_metadata)


(200149, 25)

In [8]:
data = data[['goal_pp', 'human_tactic_code', 'open_namespaces', 'decl_name', 'filename', 'split',
             *[('human_tactic_arg', i) for i in range(4)]]]
             # *[('reflected_expr_pp', i) for i in range(4)]]]
data = data.rename(columns=dict([(('human_tactic_arg', i), f'human_tactic_arg_{i}') for i in range(4)] + 
                                [(('reflected_expr_pp', i), f'reflected_expr_pp_{i}') for i in range(4)]))

/Users/Kapushev/miniconda3/lib/python3.9/site-packages/pandas/core/common.py:245: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)


## For each tactic get open local namespaces

In [9]:
data[['filename', 'row', 'col']] = [i.split(':') for i in data.index]
data = data.reset_index(drop=True)

In [10]:
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()

import sys
sys.path.append('../atp/scripts/lean/')

from lemmas_dataset import track_open_namespaces
from dependency_graph import Parser


def get_all_nested_namespaces(namespace):
    """
    Given a namespace "a.b.c" return list["a.b.c", "a.b", "a"]
    """
    if namespace is None:
        return None
    words = namespace.split('.')
    nested_namespaces = [words[0]]
    for w in words[1:]:
        nested_namespaces.append(
            f"{nested_namespaces[-1]}.{w}")
    return nested_namespaces[::-1]


def get_nonlocal_namespaces(local_namespaces, all_namespaces):
    """
    Return set difference B \ A
    """
    return np.setdiff1d(all_namespaces, local_namespaces).tolist()
    

def append_namespaces(group):
    filename = group.filename.values[0]
    with open('./_target/deps/' + filename, 'r') as fh:
        src, line_numbers = parser.remove_comments(fh.read(), lineno=True)
        
    open_namespaces = track_open_namespaces(src, line_numbers)
    out = group.copy()
    out['open_ns_local'] = [get_all_nested_namespaces(open_namespaces.get(int(row), None))
                            for row in group.row.values]
    out['open_ns_nonlocal'] = [
        get_nonlocal_namespaces(local_ns, all_ns)
        for local_ns, all_ns in zip(out.open_ns_local.values, group.open_namespaces.values)
    ]
    return out


parser = Parser()
open_namespaces_local = []

groupby = data.groupby('filename')
data = groupby.progress_apply(append_namespaces)
data.head()

  0%|          | 0/1917 [00:00<?, ?it/s]

/Users/Kapushev/miniconda3/lib/python3.9/site-packages/tqdm/std.py:814: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  return getattr(df, df_function)(wrapper, **kwargs)


,goal_pp,human_tactic_code,open_namespaces,decl_name,filename,split,human_tactic_arg_0,human_tactic_arg_1,human_tactic_arg_2,human_tactic_arg_3,row,col,open_ns_local,open_ns_nonlocal
0,"α : Type u,\n_inst_1 : inhabited α,\ni b_fst :...",unfold read read',buffer,buffer.read_eq_read',lean/library/data/buffer.lean,test,read read',,NaN,NaN,49,13,[buffer],[]
1,"α : Type u,\n_inst_1 : inhabited α,\ni b_fst :...",simp [array.read_eq_read'],buffer,buffer.read_eq_read',lean/library/data/buffer.lean,test,[array.read_eq_read'],,NaN,NaN,49,32,[buffer],[]
2,"α : Type u,\n_inst_1 : inhabited α,\nb : buffe...",cases b,buffer,buffer.read_eq_read',lean/library/data/buffer.lean,test,b,,NaN,NaN,49,4,[buffer],[]
3,"α : Type u,\ni : ℕ,\nv : α,\nb_fst : ℕ,\nb_snd...",unfold write write',buffer,buffer.write_eq_write',lean/library/data/buffer.lean,train,write write',,NaN,NaN,53,13,[buffer],[]
4,"α : Type u,\ni : ℕ,\nv : α,\nb_fst : ℕ,\nb_snd...",simp [array.write_eq_write'],buffer,buffer.write_eq_write',lean/library/data/buffer.lean,train,[array.write_eq_write'],,NaN,NaN,53,34,[buffer],[]


## 5. Load parsed lemmas

In [11]:
import json

with open(DATA_DIRECTORY / 'all_lemmas_statements.json', 'r') as fh:
    lemmas = json.load(fh)

In [12]:
# for each filename get local lemmas
import sys
sys.path.append('../atp/scripts/lean')

from lemmas_dataset import get_lemmas
from dependency_graph import Parser

lemmas_by_filename = {}
for filename in tqdm(data.filename.unique()):
    with open('./_target/deps/' + filename, 'r') as f_handle:
        src = parser.remove_comments(f_handle.read())
    lemmas_by_filename[filename] = get_lemmas(src)

  0%|          | 0/1917 [00:00<?, ?it/s]

In [13]:
import re


def check_local(lemmas_by_filename, row, cand):
    full_name = []
    
    # first check candidate names with ns prefix
    # here we also suppose that namespace are sorted
    # by nestedness --- start with the most nested
    if row.open_ns_local is not None:
        for pref in row.open_ns_local:
            if f"{pref}.{cand}" in lemmas_by_filename[row.filename]:
                full_name.append(f"{pref}.{cand}")
    # then check the candidate itself
    if cand in lemmas_by_filename[row.filename]:
        full_name.append(cand)
    return full_name
    
    
def check_nonlocal(lemmas, row, cand):
    full_names = []
    if row.open_ns_nonlocal is not None:
        for pref in row.open_ns_nonlocal:
            if f"{pref}.{cand}" in lemmas:
                full_names.append(f"{pref}.{cand}")
    if cand in lemmas:
        full_names.append(cand)
    return full_names


args_full_names = []
data['full_names_local'] = None
data['full_names_global'] = None
data['not_found_args'] = None
data['args_names'] = None
for idx, row in tqdm(data.iterrows(), total=len(data)):
    args_full_names = {'local': [], 'global': [], 'not_found': [], 'final_full_names': []}
    for i in range(4):
        args_str = row[f'human_tactic_arg_{i}']
        
        if isinstance(args_str, float) or args_str in ['only', 'with', 'in']:
            continue
            
        args_str = re.sub(r'[\[\]\(\)\{\}\<\>⟨⟩,;]', ' ', args_str)
        for cand in args_str.split():
            # skip too short names
            if len(cand) < 3:
                continue
            
            local_names = check_local(lemmas_by_filename, row, cand)
            global_names = check_nonlocal(lemmas, row, cand)
            if local_names:
                args_full_names['local'].append((cand, *local_names))
            if global_names:
                args_full_names['global'].append((cand, *global_names))
            if not local_names and not global_names:
                args_full_names['not_found'] = cand
            
            
            if local_names:
                args_full_names['final_full_names'].append(local_names[0])
            elif global_names:
                args_full_names['final_full_names'].append(global_names[0])

    data.loc[idx, 'full_names_local'] = args_full_names['local']
    data.loc[idx, 'full_names_global'] = args_full_names['global']
    data.loc[idx, 'not_found_args'] = args_full_names['not_found']
    data.loc[idx, 'args_names'] = args_full_names['final_full_names']

  0%|          | 0/200149 [00:00<?, ?it/s]

In [14]:
data.head()

,goal_pp,human_tactic_code,open_namespaces,decl_name,filename,split,human_tactic_arg_0,human_tactic_arg_1,human_tactic_arg_2,human_tactic_arg_3,row,col,open_ns_local,open_ns_nonlocal,full_names_local,full_names_global,not_found_args,args_names
0,"α : Type u,\n_inst_1 : inhabited α,\ni b_fst :...",unfold read read',buffer,buffer.read_eq_read',lean/library/data/buffer.lean,test,read read',,NaN,NaN,49,13,[buffer],[],"[(read, buffer.read), (read', buffer.read')]",[],[],"[buffer.read, buffer.read']"
1,"α : Type u,\n_inst_1 : inhabited α,\ni b_fst :...",simp [array.read_eq_read'],buffer,buffer.read_eq_read',lean/library/data/buffer.lean,test,[array.read_eq_read'],,NaN,NaN,49,32,[buffer],[],[],"[(array.read_eq_read', array.read_eq_read')]",[],[array.read_eq_read']
2,"α : Type u,\n_inst_1 : inhabited α,\nb : buffe...",cases b,buffer,buffer.read_eq_read',lean/library/data/buffer.lean,test,b,,NaN,NaN,49,4,[buffer],[],[],[],[],[]
3,"α : Type u,\ni : ℕ,\nv : α,\nb_fst : ℕ,\nb_snd...",unfold write write',buffer,buffer.write_eq_write',lean/library/data/buffer.lean,train,write write',,NaN,NaN,53,13,[buffer],[],"[(write, buffer.write), (write', buffer.write')]",[],[],"[buffer.write, buffer.write']"
4,"α : Type u,\ni : ℕ,\nv : α,\nb_fst : ℕ,\nb_snd...",simp [array.write_eq_write'],buffer,buffer.write_eq_write',lean/library/data/buffer.lean,train,[array.write_eq_write'],,NaN,NaN,53,34,[buffer],[],[],"[(array.write_eq_write', array.write_eq_write')]",[],[array.write_eq_write']


In [15]:
data = data[['goal_pp', 'human_tactic_code', 'decl_name', 'args_names', 'not_found_args']]
data = data.rename(columns={'human_tactic_code': 'tactic',
                            'not_found_args': 'other_args',
                            'goal_pp': 'goal',
                            'decl_name': 'decl'})
data.head()

,goal,tactic,decl,args_names,other_args
0,"α : Type u,\n_inst_1 : inhabited α,\ni b_fst :...",unfold read read',buffer.read_eq_read',"[buffer.read, buffer.read']",[]
1,"α : Type u,\n_inst_1 : inhabited α,\ni b_fst :...",simp [array.read_eq_read'],buffer.read_eq_read',[array.read_eq_read'],[]
2,"α : Type u,\n_inst_1 : inhabited α,\nb : buffe...",cases b,buffer.read_eq_read',[],[]
3,"α : Type u,\ni : ℕ,\nv : α,\nb_fst : ℕ,\nb_snd...",unfold write write',buffer.write_eq_write',"[buffer.write, buffer.write']",[]
4,"α : Type u,\ni : ℕ,\nv : α,\nb_fst : ℕ,\nb_snd...",simp [array.write_eq_write'],buffer.write_eq_write',[array.write_eq_write'],[]


### How to get statements

In [16]:
args_names = data.iloc[0].args_names

args_statements = [lemmas[name] for name in args_names]
for stm in args_statements:
    print(stm)

def read : Π (b : buffer α), fin b.size → α	| ⟨n, a⟩ i := a.read i
def read' [inhabited α] : buffer α → nat → α	| ⟨n, a⟩ i := a.read' i


In [23]:
data.args_names = data.args_names.apply(lambda x: '\n'.join(x))
data.other_args = data.other_args.apply(lambda x: '\n'.join(x))

data.to_parquet(DATA_DIRECTORY / 'arguments_full_names.parquet', engine='pyarrow', index=False)